In [1]:
# Import libraries, define data zones and time ranges, and authenticate
import polars as pl
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from api_wrapper.user_params import *
from api_wrapper.altitude_params import *
import api_wrapper.altitude_functions as altitude

client = altitude.get_client(db = 'example_db')

In [ ]:
# getDatabaseInfo example
params = {
"queryType": "getDatabaseInfo"
}
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, process = False)
print(job_outputs)

In [ ]:
# getExpansionFactors example
params = {
   "queryType": "getExpansionFactors",
   "year": 2021,
   "zones": richmond_zones,
   "weekdayOnly": True
   }

job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = True)
result = job_outputs[0]
expansion_factors_df = result.results

In [ ]:
# getVehicleClasses example (NOTE: requires process = True parameter)
params = {
    'queryType': 'getVehicleClasses',
    'vehicleClassSchemeId': 3
}
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = True)
result = job_outputs[0]
classes_df = result.results

In [ ]:
# getIndustries example
params = {"queryType": "getIndustries"}
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = False)
result_json = job_outputs[0]
industries_df = pl.DataFrame(result_json[0:])
# industries_df has a NAICS1From and NAICS1To column. 
# Convert to just have a NAICS column with all values in the range.
industries_df = (industries_df
    .with_columns(pl.int_ranges(pl.col("NAICS_Code_1_From"), pl.col("NAICS_Code_1_To")+1).alias("NAICS1"))
    .drop(["NAICS_Code_1_From","NAICS_Code_1_To"])
    .explode("NAICS1")
    .sort("NAICS1")
)

In [ ]:
# createZoneType example 
params = {
"queryType": "createCustomZoneType",
"comments": "Multi-county region (not necessarily contiguous).",
"customZoneTypeName": "Region"
}
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = False)
result_json = job_outputs[0]

In [ ]:
# getCustomZones example
params = {
"queryType": "getCustomZones"
}
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, out_path = out_path, serialize = True, process = False)
result_json = job_outputs[0]
custom_zones_df = pl.DataFrame(result_json[0:])

In [ ]:
# getZones example (non-custom zones only---counties in Richmond & Newport News)
params = {
    'queryType': 'getZones',
    'zoneType': 'TAZ',
    'filters':  [{'RegionIds': [entry['ZoneId'] for entry in (richmond_zones)], 'RegionType': 'County'}]
}
job_ids, job_statuses, job_outputs = altitude.query_api(client, params, process = False)
result_json = job_outputs[0]
standard_zones_df = pl.DataFrame(result_json[0:])

In [ ]:
# Option 1 to create custom zones from counties: NOT working due to API limitations, except for very small # of counties
new_zone_id = altitude.create_zone_from_counties(client, richmond_zones, "Richmond_All")
# Option 2 to create custom zones from counties: can create geojson string and upload online
new_zone_geojson = altitude.create_geojson_from_counties(client, richmond_zones)